In [1]:
from multiprocessing.pool import Pool
import numpy as np
import requests
import librosa
import json
import ray


/home/tuyendv/miniconda3/envs/e2er/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-21 16:46:57,884	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wav_path = "/data/codes/sb-apa/train/wav/chronic.wav"
transcript = "chronic"

waveform, sr = librosa.load(wav_path, sr=16000)

params = {
    "audio": waveform.tolist(),
    "transcript": transcript
}


In [3]:
inputs = [params.copy() for i in range(1)]

@ray.remote
def send_query(params):
    response = requests.post("http://14.162.145.55:9999/scoring", json=params)
    return response

outputs = ray.get([send_query.remote(params) for params in inputs])

FileNotFoundError: [Errno 2] No such file or directory: '/proc/4092506/stat'

In [ ]:
outputs[0].json()

In [ ]:
import json

json_path = [
    "/data/codes/sb-apa/train/data/scoring/apa-type-9/train.json",
    "/data/codes/sb-apa/train/data/scoring/apa-type-10/train.json",
    "/data/codes/sb-apa/train/data/scoring/apa-type-12/train.json"
]

data = {}
for path in json_path:
    _data = json.load(open(path, "r"))
    for key, value in _data.items():
        assert key not in data
        
    data.update(_data)
print(len(data))

In [ ]:
from sklearn.model_selection import train_test_split

keys = list(data.keys())

In [ ]:
train_keys, val_keys = train_test_split(keys, test_size=0.05, random_state=42)

train_set = {key:data[key] for key in train_keys}
val_set = {key:data[key] for key in val_keys}

In [ ]:
len(val_set)

In [ ]:
data_dir = "/data/codes/sb-apa/train/data/scoring"

with open(f'{data_dir}/train.json', 'w', encoding='utf-8') as f:
    json_obj = json.dumps(train_set, indent=4, ensure_ascii=False)
    f.write(json_obj)
    
with open(f'{data_dir}/val.json', 'w', encoding='utf-8') as f:
    json_obj = json.dumps(val_set, indent=4, ensure_ascii=False)
    f.write(json_obj)